In [1]:
import pandas as pd
import numpy as np 
pd.set_option('display.max_columns', 300)

## Step 1: Read in hold out data, scalers, and best model

In [2]:
holdout = pd.read_csv('data/kc_house_data_test_features.csv', index_col=0)
holdout.head()


,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,1974300020,20140827T000000,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,0,98034,47.7089,-122.241,2020,10918
1,1974300020,20150218T000000,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,0,98034,47.7089,-122.241,2020,10918
2,3630020380,20141107T000000,3,2.50,1470,1779,2.0,0,0,3,8,1160,310,2005,0,98029,47.5472,-121.998,1470,1576
3,1771000290,20141203T000000,3,1.75,1280,16200,1.0,0,0,3,8,1030,250,1976,0,98077,47.7427,-122.071,1160,10565
4,5126310470,20150115T000000,4,2.75,2830,8126,2.0,0,0,3,8,2830,0,2005,0,98059,47.4863,-122.140,2830,7916


In [3]:
import pickle

In [4]:
infile = open('pickled_files/model.pickle','rb')
model = pickle.load(infile)
infile.close()

print(model.intercept_)
print(len(model.coef_))

11.425282165459201
117


In [5]:
infile = open("pickled_files/other_info.pickle",'rb')
other_info = pickle.load(infile)
infile.close()

In [6]:
selected_cols = other_info[0]
selected_cols

Index(['sqft_living', 'sqft_lot', 'waterfront', 'view', 'grade', 'sqft_above',
       'sqft_basement', 'sqft_living15', 'sqft_lot15', 'num_times_sold',
       ...
       'zipcode_98188', 'zipcode_98198', 'zipcode_98199', 'age',
       'is_renovated', 'season_sold_Spring', 'season_sold_Summer',
       'season_sold_Winter', 'winterXsqft_living', 'springXsqft_living'],
      dtype='object', length=117)

In [7]:
holdout.isna().sum()

id               0
date             0
bedrooms         0
bathrooms        0
sqft_living      0
sqft_lot         0
floors           0
waterfront       0
view             0
condition        0
grade            0
sqft_above       0
sqft_basement    0
yr_built         0
yr_renovated     0
zipcode          0
lat              0
long             0
sqft_living15    0
sqft_lot15       0
dtype: int64

## Step 2: Feature Engineering for holdout set

Remember we have to perform the same transformations on our holdout data (feature engineering, extreme values, and scaling) that we performed on the original data.

In [8]:
holdout['date'] = pd.to_datetime(holdout['date'].str.slice(0,8),format="%Y-%m-%d")

In [9]:
dictNumTimesSold = dict(holdout.groupby('id').id.count().sort_values())
holdout['num_times_sold'] = holdout['id'].map(dictNumTimesSold)

In [10]:
holdout.drop(labels =['id', 'lat', 'long'], axis=1, inplace=True)

In [11]:
holdout['zipcode'] = holdout['zipcode'].apply(str)

In [12]:
holdout['bedrooms'] = np.where(holdout['bedrooms'] >= 7, 7, holdout['bedrooms'])

In [13]:
holdout['bathrooms'] = np.where(((holdout['bathrooms']>0)&(holdout['bathrooms']<1)), 1, holdout['bathrooms'])

holdout['bathrooms'] = np.where(holdout['bathrooms']>=5.75, 7, holdout['bathrooms'])

In [14]:
conditions = [(holdout['grade'] <= 8),
              (holdout['grade'] <= 12),
             (holdout['grade'] <= 13)]
choices = ['low','medium','high']
holdout['grade_cat'] = np.select(conditions, choices)

In [15]:
categorical = ['condition','grade_cat','bedrooms', 'bathrooms','floors','zipcode']
holdout = pd.get_dummies(holdout, columns=categorical, drop_first=True)


In [16]:
holdout['age'] = 2021 - holdout.yr_built


In [17]:
holdout['month_sold'] = holdout['date'].dt.month

In [18]:
seasons = ['Winter', 'Winter', 'Spring', 'Spring', 'Spring', 'Summer', 'Summer', 'Summer', 'Fall', 'Fall', 'Fall', 'Winter']
#{1: Winter, 2: Spring, 3: Summer, 4: Fall}
month_to_season = dict(zip(range(1,13), seasons))
holdout['season_sold'] = holdout.month_sold.map(month_to_season)

In [19]:
holdout['year_sold'] = holdout['date'].dt.year

In [20]:
holdout['is_renovated'] = np.where(holdout['yr_renovated'] > 0, 1, 0)


In [21]:
holdout = pd.get_dummies(holdout, columns = ['season_sold'], drop_first=True)
holdout['winterXsqft_living'] = holdout['season_sold_Winter']*holdout['sqft_living']
holdout['springXsqft_living'] = holdout['season_sold_Spring']*holdout['sqft_living']
holdout['ageXnumTimesSold'] = holdout['age']*holdout['num_times_sold']



In [28]:
holdout[selected_cols].head()

,sqft_living,sqft_lot,waterfront,view,grade,sqft_above,sqft_basement,sqft_living15,sqft_lot15,num_times_sold,condition_2,condition_3,condition_4,condition_5,grade_cat_low,bedrooms_1,bedrooms_2,bedrooms_3,bedrooms_4,bedrooms_5,bedrooms_6,bedrooms_7,bathrooms_1.0,bathrooms_1.5,bathrooms_1.75,bathrooms_2.0,bathrooms_2.5,bathrooms_2.75,bathrooms_3.0,bathrooms_3.25,bathrooms_3.5,bathrooms_3.75,bathrooms_4.0,bathrooms_4.25,bathrooms_4.5,bathrooms_4.75,bathrooms_5.0,bathrooms_5.25,bathrooms_5.5,bathrooms_7.0,floors_1.5,floors_2.0,floors_2.5,floors_3.0,floors_3.5,zipcode_98002,zipcode_98003,zipcode_98004,zipcode_98005,zipcode_98006,zipcode_98007,zipcode_98008,zipcode_98010,zipcode_98011,zipcode_98014,zipcode_98019,zipcode_98022,zipcode_98023,zipcode_98024,zipcode_98027,zipcode_98028,zipcode_98029,zipcode_98030,zipcode_98031,zipcode_98032,zipcode_98033,zipcode_98038,zipcode_98039,zipcode_98040,zipcode_98042,zipcode_98045,zipcode_98052,zipcode_98053,zipcode_98055,zipcode_98056,zipcode_98058,zipcode_98059,zipcode_98072,zipcode_98074,zipcode_98075,zipcode_98077,zipcode_98092,zipcode_98102,zipcode_98103,zipcode_98105,zipcode_98106,zipcode_98107,zipcode_98108,zipcode_98109,zipcode_98112,zipcode_98115,zipcode_98116,zipcode_98117,zipcode_98118,zipcode_98119,zipcode_98122,zipcode_98125,zipcode_98126,zipcode_98133,zipcode_98144,zipcode_98146,zipcode_98148,zipcode_98155,zipcode_98166,zipcode_98168,zipcode_98177,zipcode_98178,zipcode_98188,zipcode_98198,zipcode_98199,age,is_renovated,season_sold_Spring,season_sold_Summer,season_sold_Winter,winterXsqft_living,springXsqft_living
0,2270,11500,0,0,8,1540,730,2020,10918,2,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,54,0,0,1,0,0,0
1,2270,11500,0,0,8,1540,730,2020,10918,2,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,54,0,0,0,1,2270,0
2,1470,1779,0,0,8,1160,310,1470,1576,1,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,16,0,0,0,0,0,0
3,1280,16200,0,0,8,1030,250,1160,10565,1,0,1,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,45,0,0,0,1,1280,0
4,2830,8126,0,0,8,2830,0,2830,7916,1,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,16,0,0,0,1,2830,0


In [22]:
selected_cols

Index(['sqft_living', 'sqft_lot', 'waterfront', 'view', 'grade', 'sqft_above',
       'sqft_basement', 'sqft_living15', 'sqft_lot15', 'num_times_sold',
       ...
       'zipcode_98188', 'zipcode_98198', 'zipcode_98199', 'age',
       'is_renovated', 'season_sold_Spring', 'season_sold_Summer',
       'season_sold_Winter', 'winterXsqft_living', 'springXsqft_living'],
      dtype='object', length=117)

In [23]:
transformed_holdout = holdout[selected_cols]

## Step 3: Predict the holdout set

In [24]:
final_answers = np.exp(model.predict(transformed_holdout))

In [25]:
final_answers_df = pd.DataFrame(final_answers)

In [26]:
final_answers_df

,0
0,425000.722483
1,426698.104502
2,427284.396416
3,360121.127860
4,510262.681500
...,...
4317,539815.565327
4318,393649.993775
4319,334020.243759
4320,417709.463984


## Step 4: Export your predictions

In [27]:
final_answers_df.to_csv('housing_preds_jason_arikupurathu.csv')